# PCA

- Fabian Oppermann
- Petruta-Denisa Biholari
- Philipp Hasel

https://www.kaggle.com/datasets/nelgiriyewithana/most-streamed-spotify-songs-2024

In [ ]:
%pip install pandas matplotlib scikit-learn sea

In [ ]:
import pandas as pd

df = pd.read_csv("./Most Streamed Spotify Songs 2024.csv", encoding="latin1")

# 1. Datenbereinigung

In [ ]:
df = df.drop_duplicates()

# Fehlende Werte analysieren
missing = df.isnull().sum()
print("Fehlende Werte pro Spalte vor weiterer Bereinigung:")
print(missing[missing > 0])

df = df.dropna(subset=['Spotify Streams', 'TikTok Posts', 'Spotify Popularity'])

df = df.drop(columns=["TIDAL Popularity"])

df = df.drop(columns=['Track', 'Album Name', 'ISRC'])

# Datei speichern
df.to_csv("korr.csv", index=False)

# 2. kNN und PCA

In [ ]:

# Numerische Spalten identifizieren, die als Objekt-Typ geladen wurden und Kommas enthalten könnten
for col in df.columns:
    if df[col].dtype == object:
        # Versuch, Kommas zu entfernen und in numerischen Typ umzuwandeln
        # Behält Spalten, die nicht numerisch sind (z.B. 'Artist', 'Release Date'), als Objekt
        try:
            # Teste, ob nach dem Ersetzen von Kommas eine Umwandlung in float möglich ist
            pd.to_numeric(df[col].str.replace(',', '', regex=False))
            df[col] = df[col].str.replace(',', '', regex=False).astype(float)
            print(f"Spalte '{col}' wurde zu float konvertiert.")
        except ValueError:
            # Wenn die Umwandlung fehlschlägt, ist es wahrscheinlich eine echte Zeichenkette
            print(f"Spalte '{col}' konnte nicht zu float konvertiert werden und bleibt Objekt.")
        except AttributeError: # Falls .str nicht verfügbar ist (z.B. wenn es bereits numerisch ist, aber als Objekt)
             try:
                df[col] = pd.to_numeric(df[col])
                print(f"Spalte '{col}' (ursprünglich Objekt) wurde zu numerisch konvertiert.")
             except ValueError:
                print(f"Spalte '{col}' (ursprünglich Objekt) konnte nicht zu numerisch konvertiert werden.")


# Zielvariable PopularityClass erstellen mit pd.cut (konsistent)
# Die Bins sollten die gesamte mögliche Range von Spotify Popularity abdecken (typischerweise 0-100)
df['PopularityClass'] = pd.cut(df['Spotify Popularity'], 
                                bins=[-1, 40, 70, 101], # Anpassung der oberen Grenze auf 101 (oder df['Spotify Popularity'].max() + 1)
                                labels=['Low', 'Medium', 'High'], 
                                right=True) # right=True bedeutet, dass der rechte Bin-Rand inklusive ist


if df['PopularityClass'].isnull().any():
    print("Warnung: Es gibt NaN-Werte in 'PopularityClass' nach pd.cut. Überprüfen Sie die Bins und Werte in 'Spotify Popularity'.")
    print(df[df['PopularityClass'].isnull()]['Spotify Popularity'])
    # Optionale Behandlung: df.dropna(subset=['PopularityClass'], inplace=True)

In [ ]:
import numpy as np
from sklearn.calibration import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder


non_features = ['Artist', 'Release Date', 'Explicit Track', 'Spotify Popularity', 'PopularityClass']
X = df.drop(columns=non_features, errors='ignore') # errors='ignore' falls eine Spalte schon fehlt
y = df['PopularityClass'].dropna() # Sicherstellen, dass y keine NaNs hat, falls welche durch pd.cut entstanden
X = X.loc[y.index] # X und y synchron halten

# Kategoriale und numerische Features für den Preprocessor identifizieren
num_features = X.select_dtypes(include=np.number).columns
cat_features = X.select_dtypes(include=['object', 'bool']).columns # Bool hier auch als cat behandelt für OneHotEncoding

# Labels in Zahlen umwandeln für y
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Daten splitten
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

# Preprocessing Pipelines
# Numerische Pipeline: Fehlende Werte mit Median füllen (robuster gegen Ausreißer als Mittelwert) & skalieren
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")), # Geändert zu Median für Konsistenz und Robustheit
    ("scaler", StandardScaler())
])

# Kategoriale Pipeline: Fehlende Werte mit häufigstem Wert füllen & OneHot kodieren
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# ColumnTransformer, um Pipelines auf die richtigen Spalten anzuwenden
preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features)
])

# Besten Modelle laut Grid-Search:
"""
--- Random Forest ---
Beste Parameter: {'classifier__max_depth': None, 'classifier__n_estimators': 50}
Laufzeit: 1.64 Sekunden
Bestes CV-Score (Accuracy): 0.8155

--- KNN ---
Beste Parameter: {'classifier__n_neighbors': 7, 'classifier__weights': 'distance'}
Laufzeit: 0.13 Sekunden
Bestes CV-Score (Accuracy): 0.7704
"""

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


random_forest_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=50, max_depth=None, random_state=42))
])

knn_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", KNeighborsClassifier(n_neighbors=7, weights='distance'))
])

# Modelle trainieren
random_forest_pipeline.fit(X_train, y_train)
knn_pipeline.fit(X_train, y_train)

# Modelle evaluieren
from sklearn.metrics import classification_report, confusion_matrix
models = {
    "Random Forest": random_forest_pipeline,
    "kNN": knn_pipeline
}

for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    y_pred = model.predict(X_val)
    print(classification_report(y_val, y_pred, target_names=le.classes_))